In [13]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### CCXT

In [14]:
polo = ccxt.poloniex({
    'apiKey': cfg.POLONIEX_API_KEY,
    'secret': cfg.POLONIEX_API_SECRET_KEY,
})
gdax = ccxt.gdax({
    'apiKey': cfg.GDAX_API_KEY,
    'secret': cfg.GDAX_API_SECRET_KEY,
    'password': cfg.GDAX_PASSPHRASE,
    'verbose':False,
})
binance = ccxt.binance()

exchange = binance
markets = exchange.load_markets()

### Ingest Data

In [15]:
base = c.LTC
quote = c.USDT
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.BTC, c.ETH, c.LTC] ]
period = '1m' 
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [16]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period, start, end)
df.head()

Downloading: LTC/USDT
Downloaded rows: 120


,open,high,low,close,volume,time_utc
time_epoch,,,,,,
1515270660,281.19,282.24,280.56,282.23,9.59044,2018-01-06 20:31:00
1515270720,281.40,281.65,281.40,281.65,8.63000,2018-01-06 20:32:00
1515270780,281.81,282.53,281.79,282.47,4.72055,2018-01-06 20:33:00
1515270840,281.65,282.38,281.59,282.30,9.05910,2018-01-06 20:34:00
1515270900,282.26,282.97,282.26,282.97,52.62217,2018-01-06 20:35:00


In [17]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period)
df = ohlcv.load_chart_data_from_file(fpath)

In [20]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period, start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period, start)
df.head()

Downloading: BTC/USDT
Downloaded rows: 120
Downloading: ETH/USDT
Downloaded rows: 120
Downloading: LTC/USDT
Downloaded rows: 120


,BTC_USDT,ETH_USDT,LTC_USDT,time_utc
time_epoch,,,,
1515270660,16799.89,1009.61,282.23,2018-01-06 20:31:00
1515270720,16783.25,1009.75,281.65,2018-01-06 20:32:00
1515270780,16782.99,1009.54,282.47,2018-01-06 20:33:00
1515270840,16759.99,1009.57,282.30,2018-01-06 20:34:00
1515270900,16769.98,1007.50,282.97,2018-01-06 20:35:00


### Feeds

In [22]:
# CSV Feed
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)
feed.initialize()

Loading feed: data/binance_LTC_USDT_1m.csv
START 0001-01-01 01:01:00
PRIOR 0001-01-01 01:00:00


In [23]:
# Grab 1 row at a time
for i in range(3):
    data = feed.next()
    print(data['time_utc'], data['close'])

2018-01-06 20:31:00 282.23
2018-01-06 20:32:00 281.65
2018-01-06 20:33:00 282.47


In [24]:
# Access all rows in history
feed.history().head()

,open,high,low,close,volume,time_utc
time_epoch,,,,,,
1515270660,281.19,282.24,280.56,282.23,9.59044,2018-01-06 20:31:00
1515270720,281.40,281.65,281.40,281.65,8.63000,2018-01-06 20:32:00
1515270780,281.81,282.53,281.79,282.47,4.72055,2018-01-06 20:33:00
1515270840,281.65,282.38,281.59,282.30,9.05910,2018-01-06 20:34:00
1515270900,282.26,282.97,282.26,282.97,52.62217,2018-01-06 20:35:00


In [25]:
# Exchange Live Feed
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
end = None
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')

In [27]:
feed = ExchangeDataFeed(exchange, [assets[0]], period, feed_fpath, start, end)
feed.initialize()

Loading feed: data/binance_LTC_USDT_1m.csv
START 2018-01-06 21:32:38.509572
PRIOR 2018-01-06 21:31:38.509572
Downloading: BTC/USDT
Downloaded rows: 64


In [28]:
# Access History
feed.history(t_minus=3).tail()

,open,high,low,close,volume,time_utc
time_epoch,,,,,,
1515277980,17048.97,17048.98,16974.50,17019.98,5.346269,2018-01-06 22:33:00
1515278040,16962.69,17019.96,16958.31,16962.01,30.203132,2018-01-06 22:34:00
1515278100,16962.71,16962.71,16953.26,16962.00,5.786346,2018-01-06 22:35:00


In [29]:
# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = feed.next(refresh=False)
    print(data['time_utc'], data['close'])

2018-01-06 21:33:00 16680.0
2018-01-06 21:34:00 16690.01
2018-01-06 21:35:00 16690.02


In [30]:
# Refresh data before next query
data = feed.next(refresh=True)
print(data['time_utc'], data['close'])

Downloading: BTC/USDT
Downloaded rows: 61
2018-01-06 21:36:00 16690.03


In [32]:
# Example Using the Feed

def my_strategy(row, history, exchange):
    print("Time:", row['time_utc'], "Price:", row['close'])
    if row['close'] > .024:
        print("Buying coin")
        #res = exchange.limit_buy(quantity=1, price=5)
    else:
        print("Selling that stupid coin")
    return {
        'time': row['time_utc'],
        'close': row['close'],
        'action': 'buy',
        'quantity': 1,
        'price': 5,
        'exchange': exchange.id
    }

def backtest(feed, strategy, exchange):
    results = []
    row = feed.next()
    while row is not None:
        output = strategy(row, feed, exchange)
        row = feed.next(refresh=True)
        results.append(output)
    return results

In [33]:
res = backtest(feed, my_strategy, exchange)

Time: 2018-01-06 21:37:00 Price: 16689.98
Buying coin
Downloading: BTC/USDT
Downloaded rows: 59
Time: 2018-01-06 21:38:00 Price: 16700.0
Buying coin
Downloading: BTC/USDT
Downloaded rows: 58
Time: 2018-01-06 21:39:00 Price: 16700.03
Buying coin
Downloading: BTC/USDT
Downloaded rows: 57
Time: 2018-01-06 21:40:00 Price: 16759.04
Buying coin
Downloading: BTC/USDT
Downloaded rows: 56
Time: 2018-01-06 21:41:00 Price: 16730.0
Buying coin
Downloading: BTC/USDT
Downloaded rows: 55
Time: 2018-01-06 21:42:00 Price: 16769.0
Buying coin
Downloading: BTC/USDT
Downloaded rows: 54
Time: 2018-01-06 21:43:00 Price: 16768.99
Buying coin
Downloading: BTC/USDT
Downloaded rows: 53
Time: 2018-01-06 21:44:00 Price: 16790.84
Buying coin
Downloading: BTC/USDT
Downloaded rows: 52
Time: 2018-01-06 21:45:00 Price: 16802.0
Buying coin
Downloading: BTC/USDT
Downloaded rows: 51
Time: 2018-01-06 21:46:00 Price: 16770.0
Buying coin
Downloading: BTC/USDT
Downloaded rows: 50
Time: 2018-01-06 21:47:00 Price: 16793.83
Buy

### Order

In [39]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

{
    "id": "b53fe28577fb434cb8e00104ccbcebfc",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "asset": {
        "id": "LTC_USDT",
        "symbol": "LTC/USDT",
        "base": "LTC",
        "quote": "USDT"
    },
    "price": 250.0,
    "quantity": 1,
    "filled": 0,
    "order_type": "LIMIT_BUY",
    "status": "NEW",
    "created_time": "2018-01-06T22:35:52.983687",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "retries": 0
}

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics

class Strategy():
    def __init__(self, context):
        self.ctx = context
        self.initialize()
    
    def initialize(self):
        self.ctx.symbol = 'BTC/USD'
    
    def handle_data(self, data):
        # Handle next iteration of data from feed
        pass
    
class TestStrategy():
    def __init__(self):
        pass
    
    def next(self, data):
        pass

### Plotting

In [ ]:
utils.charts.plot_range(feed.history(), start, end, 'close')

### Exchange

In [ ]:
ex = TestExchange()
print(ex.fetch_balance())
orders = ex.limit_buy(c.BTC, c.USD, 1, 15000.0)
orders

In [ ]:
EX = CCXTExchange(gdax)

### Runner

In [ ]:
@unique
class TradeMode(Enum):
    BACKTEST = 0
    SIMULATE = 1
    LIVE = 2


class Punisher():
    def __init__(self, exchange, feed, strategy, record):
        self.exchange = exchange
        self.feed = feed
        self.strategy = strategy
        self.record = record
        
    def backtest(self):
        results = []
        row = self.feed.next()
        while row is not None:
            print("Timestep", row['time_utc'], "Price", row['close'])
            row = self.feed.next()
            output = self.strategy(row, self.exchange, self.feed)
            results.append(output)
        return results
    
    def simulate(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['test'].append(output)
            time.sleep(2)

    def live(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['live'].append(output)
            time.sleep(2)
    
    def punish(self, mode=False):
        if mode == TradeMode.BACKTEST:
            print("Backtesting ...")
            self.backtest()
        elif mode == TradeMode.SIMULATE:
            print("Simulating orders ...")
            self.simulate()
        elif mode == TradeMode.LIVE:
            print("LIVE TRADING! CAREFUL!")
            self.live()